In [1]:
import requests
import json
import pandas as pd
import numpy as np
import folium
import warnings
(warnings.filterwarnings('ignore'))


##the function to help user find the perfect recipe they want
def recipefinder(food, calories, amount, health_options):
    
    if health_options.lower() == "no":
        url = "https://api.edamam.com/search?q="+food+"&app_id=22fe6955&app_key=a8701478a52245bcee06a8c8333289a6&from=0&to="+amount+"&calories="+calories
        response = requests.get(url)
        recipes = response.json()
        return recipes
    
    else:
        url ="https://api.edamam.com/search?q="+food+"&app_id=22fe6955&app_key=a8701478a52245bcee06a8c8333289a6&from=0&to="+amount+"&calories="+calories+"&excluded="+health_options
        response = requests.get(url)
        recipes = response.json()
        return recipes 

def nearbygrocerystores(location):
    params = {}
    params["term"] = "grocery store"
    params["location"] =location

    token = "QpgOEI29NHp0IosYkVAneBhWrIKJBDpnfKmu1xEDua5Y1exka_FLmXcxzj9Ir7IcBtKMfqAY-QxYujYcTpRvmbeet0hzrU-KPAKD_BbrHiCdjKVVcWoD6R0D0S3vWnYx"
    headers = {'Authorization': 'Bearer %s' % token}

    url = "https://api.yelp.com/v3/businesses/search"
    request = requests.get(url, params=params, headers=headers)

    data = request.json()
    return data

In [2]:
while True:
    try:
        user_input_food = input("Please enter a ingredient you want to cook: ")
        user_input_calories = input("Please enter your desired calorie limit ex.(500): ")
        user_input_allergy = input("Are you allergic to anything? If yes, enter the thing you are allergic of. If no, just enter \"no\": ")
        user_input_amount = input("How many options do you want to see? ")
        results = recipefinder(user_input_food, user_input_calories, user_input_amount, user_input_allergy)
    #To handle errors that may occur from trying to connect to url
    except json.decoder.JSONDecodeError:
        print("Invalid inputs. Please follow the instructions and try again.")
        continue
    except requests.exceptions.RequestException as e:
        print("Error: Cannot connect. Details:", e, "Trying again...")
        continue
    except requests.exceptions.HTTPError:
        print("Response was not okay. Trying again...")
        continue
    
    #Create a list of user desired recipe and print out
    list_recipes = []
    count = 1
    print("Results: ")
    for hit in results['hits']:
        recipe_name = hit['recipe']['label']
        list_recipes.append(recipe_name)
        print(count, recipe_name)
        count += 1
    if list_recipes == []:
        print("No results found. Try again.")
        continue
    try:
        recipe_choice = input("Indicate which recipe (number) you would like more details on. If you want to restart your search, enter \"r\": ")
        if recipe_choice == 'r':
            continue
        elif int(recipe_choice) > len(list_recipes):
            print("Please choose a number within the range of provided recipes. Restarting...")
            continue
        else:
            user_choice_ingredients = input("Do you want to see the ingredients of this recipe? "
                                           "If so, enter \"Yes\", if not, enter any key to continue. ")
            if user_choice_ingredients.lower() == "yes":
                print("Ingredients:")
                print('-' * 100)
                for ingredient in results['hits'][int(recipe_choice)-1]['recipe']['ingredientLines']:
                    print("- %s" % ingredient)       
            else:
                continue
    except ValueError:
        print("Please enter an integer or 'r'. ")
        
    #create the dataframe of nutritional facts    
    user_choice_nutrition = input("Do you want to see the nutritional facts of this recipe? "
                                  "If so, enter \"Yes\", if not, enter any key to continue. ")
    if user_choice_nutrition.lower() == "yes":
        try:
            dic_list = []
            nutrients_list = ['FAT', 'SUGAR', 'PROCNT', 'CA', 'FE', 'MG', 'ZN', 'VITC', 'VITD', 'TOCPHA']
            
            '''
            #to see the whole nutrient table
            for nutrient in results['hits'][int(recipe_choice)-1]['recipe']['totalNutrients']:
                dic_list.append(results['hits'][int(recipe_choice)-1]['recipe']['totalNutrients'][nutrient])
            
            '''
            
            for nutrient in nutrients_list:
                if nutrient in results['hits'][int(recipe_choice)-1]['recipe']['totalNutrients']:
                    dic_list.append(results['hits'][int(recipe_choice)-1]['recipe']['totalNutrients'][nutrient])
                else:
                    new_dic = {}
                    new_dic['label'] = nutrient
                    new_dic['quantity'] = 0
                    new_dic['unit'] = 'none'
                    dic_list.append(new_dic)

            i=1
            nutrition_dict = {}
            for item in dic_list:
                nutrition_dict[str(i)] = pd.Series(data = item)
                i=i+1
            
            
            nutrition_df = pd.DataFrame(nutrition_dict)
            nutrition_df = nutrition_df[['1','2','3','4','5','6','7','8','9','10']]
            display(nutrition_df)
            
        except KeyError:
            print("We do not have sufficient nutritional facts to show you. Restarting...")
            continue
    else:
        pass
    print("To view the recipe, visit %s" % results['hits'][int(recipe_choice)-1]['recipe']['shareAs'])
    
    user_choice_groceries = input("Would you like to find grocery stores nearby? "
                                  "If so, enter \"Yes\", if not, enter any key to quit. ")
    if user_choice_groceries.lower() != "yes":
        print("Ending program...")
        break
    else:
        user_location = input("Where are you right now? ")

        data = nearbygrocerystores(user_location)

        grocery_stores = data['businesses']
        print("Results: ")
        
        for grocery_store in grocery_stores:
            print('-'*100)
            name = grocery_store['name']
            print(name)
            for line in grocery_store['location']['display_address']:
                print(line)
            print("%s has received a rating of %.1f stars out of 5 with %d ratings." 
                  % (name, grocery_store['rating'], grocery_store['review_count']))
            yelp_url = grocery_store['url']
            print(yelp_url)
            print('-'*100)
        print("To view these stores on a map, run the following cell.")
        break

Please enter a ingredient you want to cook: beef
Please enter your desired calorie limit ex.(500): 1000
Are you allergic to anything? If yes, enter the thing you are allergic of. If no, just enter "no": fish
How many options do you want to see? 10
Results: 
1 Roast Beef
2 Garlic Beef
3 Beef Brisket
4 Beef Kreplach
5 Beef Tacos
6 Beef Tenderloin
7 Beef Empanadas
8 Homemade Italian Beef Recipe
9 Beef Stroganoff
10 Beef Chili
Indicate which recipe (number) you would like more details on. If you want to restart your search, enter "r": 1
Do you want to see the ingredients of this recipe? If so, enter "Yes", if not, enter any key to continue. yes
Ingredients:
----------------------------------------------------------------------------------------------------
- 8-lb. shell roast of beef
- Coarse salt and freshly ground black pepper
Do you want to see the nutritional facts of this recipe? If so, enter "Yes", if not, enter any key to continue. yes


,1,2,3,4,5,6,7,8,9,10
label,Fat,Sugars,Protein,Calcium,Iron,Magnesium,Zinc,VITC,VITD,Vitamin E
quantity,441.247,0.0696718,752.643,632.873,67.8815,817.107,138.038,0,0,13.1767
unit,g,g,g,mg,mg,mg,mg,none,none,mg


To view the recipe, visit http://www.edamam.com/recipe/roast-beef-3921adf30bb0c9736b9ac30f447f8a63/beef/1000-cal
Would you like to find grocery stores nearby? If so, enter "Yes", if not, enter any key to quit. yes
Where are you right now? Syracuse University
Results: 
----------------------------------------------------------------------------------------------------
Wegmans
3325 W Genesee St
Syracuse, NY 13219
Wegmans has received a rating of 4.0 stars out of 5 with 29 ratings.
https://www.yelp.com/biz/wegmans-syracuse-3?adjust_creative=nd8E-ENnmQPB7dGmYrkHZw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=nd8E-ENnmQPB7dGmYrkHZw
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Samir's Imported Foods
811 E Genesee St
Syracuse, NY 13210
Samir's Imported Foods has received a rating of 4.5 stars out of 5 with 14 rati

In [4]:
# data = nearbygrocerystores(user_location)
center = (data['businesses'][0]['coordinates']['latitude'], data['businesses'][0]['coordinates']['longitude'])

map = folium.Map(location = center, zoom_start = 11)

grocery_stores = data['businesses']

for grocery_store in grocery_stores:
    lat = grocery_store['coordinates']['latitude']
    long = grocery_store['coordinates']['longitude']
    pos = (lat, long)
    annotation = folium.Popup(grocery_store['name'], parse_html=True)
    marker = folium.Marker(location = pos, popup = annotation)
    map.add_children(marker)
map